# **<span style="color:red">Imports</span>**

In [1]:
# Math
import math

# Handy arrays
import numpy as np

# DeepXDE
import deepxde as dde
import torch

Using backend: pytorch
Other supported backends: tensorflow.compat.v1, tensorflow, jax, paddle.
paddle supports more examples now and is recommended.


# <span style="color:red">**Постановка**</span>

<div>
<img src="var-07.png" width="600"/>
</div>

# <span style="color:red">**PINN**</span>

## <span style="color:orange">**Init**</span>

### <span>**Функция вычисления невязок**</span>

In [2]:
def pde(domain: torch.Tensor, u: torch.Tensor) -> torch.Tensor:
    print("pde")
    # d^2 u
    # -----
    # dx ^2
    du_xx = dde.grad.hessian(u, domain, i=0, j=0)

    # d^2 u
    # -----
    # dy ^2
    du_yy = dde.grad.hessian(u, domain, i=1, j=1)

    # du/dt
    du_t = dde.grad.jacobian(u, domain, j=2)

    # Equation
    return (du_t - 2.3 * (du_xx + du_yy)
            - 2.5586 * torch.exp(-0.7 * domain[:, 2]) * torch.sin(0.9 * domain[:, 0])
            - 13.5616 * torch.exp(-0.7 * domain[:, 2]) * torch.cos(2.2 * domain[:, 1]))

### <span>**Геометрия**</span>

In [3]:
# Boundaries
x_left, x_right = 0, math.pi
y_down, y_up = -math.pi, math.pi
T0, T1 = 0, 1

# X & Y geomentry (rectangle)
geometry = dde.geometry.Rectangle(xmin=[x_left, y_down], xmax=[x_right, y_up])
# Time segment [0, 1]
time_domain = dde.geometry.TimeDomain(T0, T1)
# Final domain
domain = dde.geometry.GeometryXTime(geometry, time_domain)

### <span>**Функции принадлежности точки границе**</span>

In [4]:
# Left boundary
def l_boundary(domain, on_boundary):
    return on_boundary and dde.utils.isclose(domain[0], x_left)

In [5]:
# Right boundary
def r_boundary(domain, on_boundary):
    return on_boundary and dde.utils.isclose(domain[0], x_right)

In [6]:
# Up boundary
def up_boundary(domain, on_boundary):
    return on_boundary and dde.utils.isclose(domain[1], y_down)

In [7]:
# Down boundary
def down_boundary(domain, on_boundary):
    return on_boundary and dde.utils.isclose(domain[1], y_up)

In [8]:
# Initial boundary
def initial_boundary(domain, on_initial):
    return on_initial and dde.utils.isclose(domain[2], T0)

### <span>**Граничные и начальные условия**</span>

In [9]:
# Constant to train
c_lambda = dde.Variable(2.0)

In [10]:
# Boundary condition left
def bc_l_func(domain):
    # Get domain axis
    x,y,t = domain

    return 1.98 * torch.exp(-0.7 * t)

bc_l = dde.NeumannBC(domain, bc_l_func, l_boundary)

In [11]:
# Boundary condition right
def bc_r_func(domain):
    # Get domain axis
    x,y,t = domain
    
    return 1.3 + 1.3 * torch.exp(-0.7 * t) * torch.cos(2.2 * y) - 0.55 * (1 - math.sqrt(5)) * torch.exp(-0.7 * t)

bc_r = dde.DirichletBC(domain, bc_r_func, r_boundary)

In [12]:
# Boundary condition up
def bc_up_func(domain):
    # Get domain axis
    x,y,t = domain
    
    return -2.86 * math.sqrt(0.625 - math.sqrt(5)/8) * torch.exp(-0.7 * t)

bc_up = dde.DirichletBC(domain, bc_up_func, up_boundary)

In [13]:
# Boundary condition down
def bc_down_func(domain):
    # Get domain axis
    x,y,t = domain
    
    return 1.3 + 2.2 * torch.exp(-0.7 * t) * torch.sin(0.9 * x) + 0.875 * (1 + math.sqrt(5)) * torch.exp(-0.7 * t)

bc_down = dde.NeumannBC(domain, bc_down_func, down_boundary)

In [14]:
# Initial condition
def ic_func(domain):
    # Get domain axis
    x,y,t = domain
    
    return c_lambda + 2.2 * torch.sin(0.9 * x) + 1.3 * torch.cos(2.2 * y)

ic = dde.IC(domain, ic_func, initial_boundary)

In [15]:
# Observed data
data = np.load('var-07.npz')
ob_domain = data['xyt']
ob_u = data['u']
observed_u = dde.icbc.PointSetBC(ob_domain, ob_u, component=0)

### <span>**Система уравнений**</span>

In [16]:
data = dde.data.TimePDE(domain, pde, [bc_l, bc_r, bc_up, bc_down, ic, observed_u],
                        num_domain=10, num_boundary=10, num_initial=10,
                        anchors=ob_domain,
                        num_test=100)

### <span>**Нейронная сеть**</span>

In [17]:
layer_size = [3] + [50] * 2 + [1]

net = dde.maps.FNN(layer_size, "tanh", "Glorot uniform")
net.apply_output_transform(lambda x, y: abs(y))

### <span>**Обёртка нейронной сети**</span>

In [18]:
# Epochs and batch size
iterations = 1000
batch_size = 16

# Init and compile model
model = dde.Model(data, net)
model.compile("adam", lr=1e-3, loss_weights=[1, 1, 1, 1, 1, 1])

Compiling model...
'compile' took 2.529178 s



In [19]:
# Callbacks
checker = dde.callbacks.ModelCheckpoint(
    "model/model1.ckpt", save_better_only=True, period=iterations/10
)
variable = dde.callbacks.VariableValue(c_lambda, period=iterations/10)

In [20]:
# Train
losshistory, trainstate = model.train(iterations=iterations, callbacks = [variable])

Training model...

pde


OutOfMemoryError: CUDA out of memory. Tried to allocate 65.05 GiB. GPU 0 has a total capacity of 16.00 GiB of which 14.34 GiB is free. Of the allocated memory 496.33 MiB is allocated by PyTorch, and 25.67 MiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)

In [ ]:
# Finish training with different optimizer
model.compile("L-BFGS-B")
losshistory, train_state = model.train(iterations=iterations, batch_size=batch_size)

In [ ]:
# Save training plot
dde.saveplot(losshistory, trainstate, issave=True, isplot=True)

Compiling model...
'compile' took 0.000621 s

Training model...

